In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
from collections import defaultdict
from collections import Counter
import csv
from clusim.clustering import Clustering, print_clustering
import clusim.sim as sim
import math
from multiprocessing import Pool
from tqdm import tqdm
import itertools

In [ ]:
def get_nmi_subset(target_dict, ref_dict):
    c1_name = set(target_dict)
    c2_name = set(ref_dict)
    intersectionset = c1_name.intersection(c2_name)
    del(c1_name)
    del(c2_name)
    dict1 = {}
    dict2 = {}
    for idx_tmp, val in enumerate(intersectionset):
        dict1[idx_tmp] = [target_dict[val]]
        dict2[idx_tmp] = [ref_dict[val]]
    c1 = Clustering(elm2clu_dict = dict1)
    c2 = Clustering(elm2clu_dict = dict2)
    nmi_sim = sim.nmi(c1, c2)
    return nmi_sim

def get_params_from_filename(fname):
    subname = fname.split("/")[3]
    net_type = subname.split("_", 1)[0]
    res = subname.rsplit("_", 1)[1].rsplit(".", 1)[0]
    temp = subname.split("_", 3)
    if(temp[2][0]=="r"):
        fieldname = temp[1]
        r_layer = tuple(temp[3].rsplit("_", 1)[0].split("_"))
    else:
        fieldname = temp[1]+" "+temp[2]
        r_layer = tuple(temp[3].split("_", 1)[1].rsplit("_", 1)[0].split("_"))
    return [net_type, fieldname, r_layer, res]

def calc_granularity_cludict(cluster_dict):
    # cluster list: list idx = cluster idx
    # list[cluster idx]: list of node indexes for a certain cluster
    cluster_count = len(set(cluster_dict.values()))
    total_count = len(cluster_dict)
    gran_denom = sum([x**2 for x in Counter(cluster_dict.values()).values()])
    gran1 = total_count / gran_denom
    #gran2 = len(cluster_count) / gran_denom
    return gran1

def get_nmi_file(fname):
    param_list = get_params_from_filename(fname)
    clusters = pd.read_csv(fname, sep="\t", header=None, names=["PaperId", "ClusterNum"])
    L0FOSID = fos_l0_namedict[param_list[1]]
    clusterdict = clusters.set_index("PaperId")["ClusterNum"].to_dict()
    nmi_val = get_nmi_subset(clusterdict, paperfielddict)
    gran = calc_granularity_cludict(clusterdict)
    res = param_list + [gran, nmi_val]
    return res

In [ ]:
#FieldReferences = pd.read_csv("./Processed_data/MAG_ALLReferenceL0FieldMerged.tsv", sep="\t", )
#FieldReferences

In [ ]:
FOS_datafile = "./MAG_dataset/advanced/FieldsOfStudy.txt"
df_fosinfo = pd.read_csv(FOS_datafile, sep="\t", header=None, usecols=[0, 3, 5, 6], quoting=csv.QUOTE_NONE)
df_fosinfo.columns = ["FieldofStudyId", "FosName", "FosLevel", "PaperCount"]
fos_l0_namedict = df_fosinfo[df_fosinfo["FosLevel"] == 0].set_index("FosName")["FieldofStudyId"].to_dict()

In [ ]:
input_file_mag_l2_field = "./fastnvme01//MAG_PaperFieldL2_MAX.txt"
paperfielddict = defaultdict(int)
with open(input_file_mag_l2_field, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    next(reader)
    for row in reader:  
        paperfielddict[int(row[0])] = int(row[3])
print(len(paperfielddict))

In [ ]:
no_process = 8
filenames = sorted(glob.glob("./fastnvme01/Cluster_out/DC-T1_Physics_r_1.00_0.00_0.00_*.clu"))

In [ ]:
#filenames

In [ ]:
no_process = 10

pool = Pool(processes=no_process)
sub_res_list = list(tqdm(pool.imap(get_nmi_file, filenames), total=len(filenames)))
pool.close()

In [ ]:
result_list = []
result_list.append(["NetworkType", "L0FOS", "LayerRatio", "ResParm", "Granularity", "L2NMI"])
result_list += sub_res_list

In [ ]:
df_l2_sim_Phyiscs_T1 = pd.DataFrame(result_list)
df_l2_sim_Phyiscs_T1.columns = df_l2_sim_Phyiscs_T1.iloc[0]
df_l2_sim_Phyiscs_T1 = df_l2_sim_Phyiscs_T1[1:]
df_l2_sim_Phyiscs_T1["LayerRatio"] = df_l2_sim_Phyiscs_T1["LayerRatio"].astype(str)
df_l2_sim_Phyiscs_T1.to_csv("./MAG_L2_Similarity/20210423_DC_3.tsv", sep = "\t", index = None)

In [ ]:
df_l2_sim_Phyiscs_T1